05월 08일 화요일 : 실습
===
이제까지 배운 function 의 개념을 이용하여, 실습을 통해 적용해보자!

In [24]:
class Machine():
    def __init__(self, x_speed, y_speed):
        self.x_pos = -1
        self.y_pos = -1
        self.x_speed = x_speed
        self.y_speed = y_speed
    
    def move_to(self, x_pos, y_pos):
        self.x_pos = x_pos
        self.y_pos = y_pos
    
    def move_time(self, x_pos, y_pos):
        time = abs(x_pos - self.x_pos)/self.x_speed
        time += abs(y_pos - self.y_pos)/self.y_speed
        return time
    
    def move_to_init(self):
        self.move_to(-1,-1)
        
    def __str__(self):
        return "{}, {}".format(self.x_pos, self.y_pos)

In [25]:
class Rack():
    def __init__(self, x_size, y_size):
        self.x_size = x_size
        self.y_size = y_size
        self.cells = {}
        for x in range(x_size):
            for y in range(y_size):
                self.cells[(x,y)] = "X"
        
    def __str__(self):
        result_str = ""
        for y in range(self.y_size)[::-1]:
            y_cells_list = [self.cells[(x,y)] for x in range(self.x_size)]
            result_str += " ".join(y_cells_list) + "\n"
        return result_str

In [26]:
a = Rack(5,6)
print(a)

X X X X X
X X X X X
X X X X X
X X X X X
X X X X X
X X X X X



In [27]:
class ASRS_Rack(Rack):
    def __init__(self, x_size, y_size):
        super().__init__(x_size,y_size)
        
    def search_product(self, product_name):
        result_list = [position for position in self.cells.keys() if self.cells[position] == product_name]
        return result_list

In [13]:
rack1 = ASRS_Rack(3,4)
print(rack1)
rack1.search_product("X")

X X X
X X X
X X X
X X X



[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3)]

In [31]:
class Crane(Machine):
    def __init__(self, x_speed, y_speed, shuttle_size):
        super().__init__(x_speed,y_speed)
        self.shuttle_size = shuttle_size
        self.shuttle = ["X"] * shuttle_size
        
    def has_product(self, product_name):
        for idx in range(self.shuttle_size):
            if self.shuttle[idx] == product_name:
                return idx
    
    def store(self, rack, product_name):
        if self.has_product(product_name):
            self.shuttle[self.has_product(product_name)] = "X"
            rack.cells[(self.x_pos, self,y_pos)] = product_name
            print("product {} is stored at ({},{})!".format(product_name, self.x_pos, self.y_pos))
        
    def retrieve(self, rack):
        if self.has_product("X"):
            self.shuttle[self.has_product("X")] = rack.cells[(self.x_pos, self.y_pos)]
            rack.cells[(self.x_pos, self.y_pos)] = "X"
            print("product {} is retrieved from ({},{})!".format(self.shuttle[self.has_product("X")], self.x_pos, self.y_pos))
    
    def load(self, product_name):
        if self.has_product("X"):
            self.shuttle[self.has_product("X")] = product_name
            print("product {} is loaded!".format(product_name))
    
    def release(self):
        self.shuttle = ["X" for i in range(self.shuttle_size)]
        print("all product are released!")
    
    def __str__(self):
        return "crate at ({},{}) with {}".format(self.x_pos, self.y_pos, self.shuttle)

In [32]:
crane1 = Machine(2.0, 3.0)
crane1 = Crane(1,2,3)
print(crane1)

crate at (-1,-1) with ['X', 'X', 'X']


In [41]:
class ASRS_system():
    def __init__(self, rack, crane):
        self.rack = rack
        self.crane = crane
    
    def start_cycle(self, names, srs):
        if "S" in srs:
            print(self.crane)
            for idx in range(len(srs)):
                if srs[idx] == "S":
                    self.crane.load(names[idx])
            print(self.crane)
                
    def end_cycle(self, srs):
        self.crane.move_to_init()
        if "R" in srs:
            print(self.crane)
            self.crane.release()
            print(self.crane)
    
    def operate_cycle(self, names, srs):
        for idx in range(self.crane.shuttle_size):
            if srs[idx] == "S":
                cells_pos = self.rack.search_product("X")[0]
                self.crane.move_to(cells_pos[0], cells_pos[1])
                self.crane.store(self.rack, names[idx])
            if srs[idx] == "R":
                cells_pos = self.rack.search_product(names[idx])[0]
                self.crane.move_to(cells_pos[0], cells_pos[1])
                self.crane.retrieve(self.rack)
                
    def operate_system(self, names, srs):
        min_len = min(len(names), len(srs))
        num_cycles = min_len // self.crane.shuttle_size
        
        min_len = num_cycles * self.crane.shuttle_size
        names, srs = names[:min_len], srs[:min_len]
        
        print(self.rack)
        for cycle_idx in range(num_cycles):
            print(cycle_idx)
            seq_idx = cycle_idx * self.crane.shuttle_size
            cycle_names = names[seq_idx : seq_idx + self.crane.shuttle_size]
            cycle_srs = srs[seq_idx : seq_idx + self.crane.shuttle_size]
            
            self.start_cycle(cycle_names, cycle_srs)
            self.operate_cycle(cycle_names, cycle_srs)
            self.end_cycle(cycle_srs)
            print(self.rack)